The goal of this notebook is to find out the performance of the 5mo model when trying to predict the remaining 9mo. If it isn't very good, then the idea is to use this model to initialize a LwF model and use that to improve performance.

In [3]:
from dataclasses import dataclass
from typing import Union

@dataclass
class DatasetConfiguration:
    users: Union[None, list] = None
    dataset: str = 'logs-14mo.pkl'
    keep_username: bool = False
    keep_timestamp: bool = False

In [4]:
DatasetConfiguration()

DatasetConfiguration(users=None, dataset='logs-14mo.pkl', keep_username=False, keep_timestamp=False)